### 1. Feature Engineering

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

parser = lambda date: pd.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
# energydata_df=pd.read_csv("energydata_complete.csv",parse_dates=['date'], date_parser=parser)
nrgconsumption=pd.read_csv("energydata_complete.csv",index_col=[0],parse_dates=['date'], date_parser=parser)

pd.set_option('display.max_columns', 30)
nrgconsumption.head()


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097



The outside temperature determines as to increase/decrease the heat of the house. It would be useful to know the lowest temperature within the day in order to figure out at what time the consumption is great. So, one feature we can add is the **lowest temperature per day** along with the **hour of the day**.


In [2]:

nrgconsumption['Hour_of_day']=nrgconsumption.index.map(lambda x: pd.to_datetime(x).hour)
nrgconsumption['Lowest_T_out_of_day']=nrgconsumption.groupby(nrgconsumption.index.dayofyear)['T_out'].transform(min)


The information if it is a **business day or holiday**, it is a factor that affects the electric consumption.

**Seasons** is a feature that could reveal different patterns in electric appliances use in household.

In [3]:

seasons = {'spring' : [3, 5],'summer' : [6, 8],'fall' : [9, 11]}

def get_season(date): 
    month=date.month
    if month in seasons['spring']:
        return 0
    if month in seasons['summer']:
        return 1
    if month in seasons['fall']:
        return 2
    else:
        return 3
        
nrgconsumption['Season']=nrgconsumption.index.map(get_season)


* The information if it it weekday or weekend is useful

In [4]:
nrgconsumption['weekday']=nrgconsumption.index.map(lambda x: 0 if x.weekday()<5 else 1)

* In order to use the outside humidity along with the outside temperature we can compute an additional feature the 'heat_index' since the 2 features are high autocorrelated and by adding both as candidate variables will not give value at our model. We will use the below function to calculate the apparent temperature based on air temperature and relative humidity. The apparent temperature is often described as how hot it feels to the human body.

In [5]:
def calculate_heat_index(T,H):
    return (-42.379 + (2.04901523 * T) + (10.14333127 * H) - (0.22475541 * T * H)  - (6.83783e-3 * T * T) - (5.481717e-2 * H * H) + (1.22874e-3 * T * T * H) + (8.5282e-4 * T * H * H) - (1.99e-6 *T*T*H*H))

nrgconsumption['heat_index'] = calculate_heat_index(nrgconsumption['T_out'], nrgconsumption['RH_out'])


* In order to avoid using in our model all the features about temperature and humidity for every room, we can compute the average row-wise for all temperatures and the average for all the humidity values.

In [6]:
nrgconsumption['avgT_interior'] = nrgconsumption[['T1','T2','T3','T4','T5','T6','T7','T8','T9']].mean(axis=1)
nrgconsumption['avgRH_interior'] = nrgconsumption[['RH_1','RH_2','RH_3','RH_4','RH_5','RH_6','RH_7','RH_8','RH_9']].mean(axis=1)

**Also we should check if tha dataset has any null values to remove because it will affect our prediction**

In [34]:
nrgconsumption.isnull().values.any()

False

### 2. Train 3 different models on the data

* Let's start with the **simple linear regression model.**

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.grid_search import GridSearchCV


C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [36]:
X = pd.DataFrame(nrgconsumption['heat_index'])
y = pd.DataFrame(nrgconsumption['Appliances'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [37]:
# 1. We set up the model
model = LinearRegression()
# 2. We fit it
model.fit(X_train, y_train)
# 3. We check the score
model.score(X_test, y_test)

0.014340243275344111

* Then will follow the **SVR model** and we will use multiple independent variables to train the model.

In [38]:
X =  pd.DataFrame({"heat_index": nrgconsumption.heat_index,"avgT_interior":nrgconsumption.avgT_interior,
                  "avgRH_interior":nrgconsumption.avgRH_interior,"Season":nrgconsumption.Season, 
                   "Hour_of_day":nrgconsumption.Hour_of_day,"weekday":nrgconsumption['weekday']},index=nrgconsumption.index)

y = pd.DataFrame(nrgconsumption['Appliances'])

In [39]:
# 1. We set up the model
svr_model = svm.SVR()
# 2. We fit it
svr_model.fit(X, y) 
# 3. We check the score
svr_model.score(X, y)

C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-0.044558235613409984

* Then follows the **Random Forest** model

In [8]:
X =  pd.DataFrame({"heat_index": nrgconsumption.heat_index,"avgT_interior":nrgconsumption.avgT_interior,
                  "avgRH_interior":nrgconsumption.avgRH_interior,"Season":nrgconsumption.Season, 
                   "Hour_of_day":nrgconsumption.Hour_of_day,"weekday":nrgconsumption['weekday']},index=nrgconsumption.index)

y = pd.DataFrame(nrgconsumption['Appliances'])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [11]:
# # We instantiate the model with 100 decision trees
rf_model = RandomForestRegressor(n_estimators = 100)

rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_test)
print(y_pred)
from sklearn.metrics import r2_score
r2_score(y_test , y_pred)

C:\Users\Kate\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


[329.1  45.9 316.1 ...  46.  312.6  99.7]


0.4795060559374914

* As accuracy measure we will use the model.score() which is the coefficient of determination. It takes a feature matrix X_test and the expected target values y_test and the predictions for X_test are compared with y_test.


### **3. Model Evaluation & Selection**

* We will apply as a cross validation strategy the K-Fold Cross Validation. We will split into a 4 sections/folds where each fold is used as a testing set.

* First, we will perform K-Fold CV on a **simple linear regression model**.

In [43]:
X = pd.DataFrame(nrgconsumption['heat_index'])
y = pd.DataFrame(nrgconsumption['Appliances'])
model = LinearRegression()
scores = []
kfold = KFold(n_splits=4, shuffle=True, random_state=4)
for i, (train, test) in enumerate(kfold.split(X, y)):
    model.fit(X.iloc[train,:], y.iloc[train,:])
    score = model.score(X.iloc[test,:], y.iloc[test,:])
    scores.append(score)
print(np.mean(scores))

0.013787464070406014


* Then , we will perform K-Fold CV on **SVR model**.

In [44]:
X =  pd.DataFrame({"heat_index": nrgconsumption.heat_index,"avgT_interior":nrgconsumption.avgT_interior,
                  "avgRH_interior":nrgconsumption.avgRH_interior,"Season":nrgconsumption.Season, 
                   "Hour_of_day":nrgconsumption.Hour_of_day,"weekday":nrgconsumption['weekday']},index=nrgconsumption.index)

y = pd.DataFrame(nrgconsumption['Appliances'])

In [45]:
scores = []
svr = svm.SVR()
cv = KFold(n_splits=4, random_state=0, shuffle=False)
for train_index, test_index in cv.split(X):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    svr.fit(X_train, y_train)
    scores.append(svr.score(X_test, y_test))

print(np.mean(scores))

Train Index:  [ 4934  4935  4936 ... 19732 19733 19734] 

Test Index:  [   0    1    2 ... 4931 4932 4933]


C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train Index:  [    0     1     2 ... 19732 19733 19734] 

Test Index:  [4934 4935 4936 ... 9865 9866 9867]
Train Index:  [    0     1     2 ... 19732 19733 19734] 

Test Index:  [ 9868  9869  9870 ... 14799 14800 14801]
Train Index:  [    0     1     2 ... 14799 14800 14801] 

Test Index:  [14802 14803 14804 ... 19732 19733 19734]
-0.09604180095024523


* We perform Cross-Validation with a **Random Forest** model

In [12]:
cv_r2 = cross_val_score(estimator = rf_model, X = x_train, y = y_train, cv = 10, scoring='r2')
cv_r2.mean()

C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was pa

0.5140559722241268

 * We fit the training set almost perfectly with R-squared score at 0.92

In [13]:
train_pred = rf_model.predict(x_train)
r2_score(y_train , train_pred)

0.9346204311652275

* But, as always, the moment of truth comes when we apply the model on the test set

In [14]:
r2_score(y_test , y_pred)

0.4795060559374914

### 4. Parameter Tuning

In [15]:
X = pd.DataFrame({"heat_index": nrgconsumption.heat_index,"avgT_interior":nrgconsumption.avgT_interior,
                  "avgRH_interior":nrgconsumption.avgRH_interior,"Season":nrgconsumption.Season, 
                   "Hour_of_day":nrgconsumption.Hour_of_day,"weekday":nrgconsumption['weekday']},index=nrgconsumption.index)

y = pd.DataFrame(nrgconsumption['Appliances'])

In [16]:
rf_model = RandomForestRegressor(n_estimators = 1, random_state = 0)

param_grid = {
                 'n_estimators': [10, 20, 50, 100],
                 'max_depth': [2, 5, 7, 9]
             }

In [ ]:
from sklearn.grid_search import GridSearchCV

grid_clf = GridSearchCV(rf_model, param_grid, cv=4)
grid_clf.fit(X, y)
grid_clf.best_estimator_

C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1675: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1675: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1675: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1675: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1675: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1675: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1675: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1675: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas